In this project, we're going to practice using our SQL skills to answer business questions.

The databse is called Chinook, which represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Let's start by getting familiar with our data. Remember that we can query the database to get a list of all tables and views in our database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Write one or two queries to get familiar with the tables and to practice running SQL in this interface. Use the schema diagram on the previous screen for reference.

In [3]:
%%sql
SELECT *
FROM employee
LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. The four genres are `Hip-Hop`, `Punk`, `Pop`, and `Blues`.

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We will write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [4]:
%%sql
WITH genre_sold AS
    (
    SELECT il.track_id AS track_id,
           t.genre_id AS genre_id,
           g.name AS name,
           SUM(il.quantity) AS sold
    FROM invoice_line AS il
    INNER JOIN track AS t ON t.track_id = il.track_id
    INNER JOIN genre AS g ON g.genre_id = t.genre_id
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    GROUP BY g.name
    ORDER BY sold DESC
    )

SELECT name,
       sold,
       CAST(sold AS Float) / (SELECT SUM(sold) FROM genre_sold) AS sold_pct
FROM genre_sold;

 * sqlite:///chinook.db
Done.


name,sold,sold_pct
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the four genres we have: `Hip-Hop`, `Punk`, `Pop`, and `Blues`, and given the sales data, we will recommend for `Punk`, `Blues` and `Pop` as the three albums we should purchase.

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We will write a query that finds the total dollar amount of sales assigned to each sales support agent. We will also add any extra attributes that we may find relevant to the analysis.

In [5]:
%%sql
WITH employee_data AS
    (
    SELECT *
    FROM employee AS e
    LEFT JOIN customer AS c ON c.support_rep_id = e.employee_id
    LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
    )
    
SELECT
    first_name,
    last_name,
    title,
    SUM(total) AS total_sales
FROM employee_data
GROUP BY employee_id
ORDER BY total_sales DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


first_name,last_name,title,total_sales
Jane,Peacock,Sales Support Agent,1731.510000000004
Margaret,Park,Sales Support Agent,1584.0000000000032
Steve,Johnson,Sales Support Agent,1393.9200000000028
Andrew,Adams,General Manager,None
Nancy,Edwards,Sales Manager,None
Michael,Mitchell,IT Manager,None
Robert,King,IT Staff,None
Laura,Callahan,IT Staff,None


From the data above, we notice that there are only three employees with sales. Does it mean that only three employees are doing their job?

In [6]:
%%sql
SELECT
    first_name,
    last_name
FROM employee
WHERE title = 'Sales Support Agent';

 * sqlite:///chinook.db
Done.


first_name,last_name
Jane,Peacock
Margaret,Park
Steve,Johnson


From above, we have confirmed that in the whole company, only three of them are doing sales. Hence, it make sense that only three of them will have sales data.

We will delve deeper to see if there are any other relevant attributes contributing to higher sales.

In [7]:
%%sql
SELECT
    first_name,
    last_name,
    reports_to,
    hire_date
FROM employee
WHERE title = 'Sales Support Agent';

 * sqlite:///chinook.db
Done.


first_name,last_name,reports_to,hire_date
Jane,Peacock,2,2017-04-01 00:00:00
Margaret,Park,2,2017-05-03 00:00:00
Steve,Johnson,2,2017-10-17 00:00:00


From earlier analysis, we see that `Jane` has the highest sales, followed by `Margaret` and `Steve`.

From this analysis, we see that all three of them report to the same supervisor. However, `Jane` who was hired earlier generated the highest sale. This is logical as she had been in her job for the longest doing sales.

The next task is to analyze the sales data for customers from each different country.

In particular, we will calculate data, for each country, on the:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, we will group these customers as "Other" in the analysis.

In [49]:
%%sql
WITH country_count AS
    (
    SELECT
        CASE
        WHEN (SELECT COUNT(*)
              FROM customer
              WHERE country = c.country) = 1 THEN 'Other'
        ELSE c.country
        END AS country,
    c.customer_id,
    il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    total_customers,
    total_sales,
    customer_lifetime_value,
    average_order
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) AS total_customers,
        SUM(unit_price) AS total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) AS customer_lifetime_value,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) AS average_order,
        CASE
        WHEN country = 'Other' THEN 1
        ELSE 0
        END AS sort
    FROM country_count
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );



 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,customer_lifetime_value,average_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


We have been asked to find out what percentage of purchases are individual tracks vs whole albums.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [69]:
%%sql
SELECT track_id
FROM track
WHERE album_id = 91

 * sqlite:///chinook.db
Done.


track_id
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167


In [106]:
%%sql


WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id AS invoice_id,
        MIN(il.track_id) AS first_track_id
    FROM invoice_line AS il
    GROUP BY invoice_id
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    CAST(COUNT(invoice_id) AS Float) / (SELECT COUNT(*) FROM invoice) AS percent
FROM
    (
    SELECT
        *,
        CASE
        WHEN
        (
        SELECT track_id 
        FROM track
        WHERE album_id = (SELECT album_id FROM track
                          WHERE track_id = ifs.first_track_id)

        EXCEPT

        SELECT track_id
        FROM invoice_line
        WHERE invoice_id = ifs.invoice_id
        ) IS NULL

        AND

        (
        SELECT track_id 
        FROM invoice_line
        WHERE invoice_id = ifs.invoice_id

        EXCEPT

        SELECT track_id 
        FROM track
        WHERE album_id = (SELECT album_id FROM track
                          WHERE track_id = ifs.first_track_id)
        ) IS NULL
        THEN 'Yes'
        ELSE 'No'
        END AS 'album_purchase'
    FROM invoice_first_track AS ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674
